In [ ]:
import os
import glob
import shutil
import numpy as np
import casadi as ca

In [ ]:
# Problem configuration
start = [0.0, 0.0, 0.0]
goal = [0.0, -5.0, -np.pi/2]
min_dist_to_goal = 0.5

wheel_radius = 0.016 # m
distance_between_wheels = 0.089 # m
max_vel_leftwheel = 157.08 # rad/s
min_vel_leftwheel = 15 # rad/s
max_vel_rightwheel = 77.08 # rad/s
min_vel_rightwheel = 15 # rad/s
time_step = 0.01 # s

In [ ]:
class TO_MPC():
    def __init__(self, config_file_path=None):
        # Parameters
        self.N = 20
        self.dt = time_step
        self.nx = 3
        self.nu = 2

        # State limits
        x_min = [-20.0, -20.0, -ca.inf]
        x_max = [20.0, 20.0, ca.inf]

        # Control limits
        u_min = [min_vel_leftwheel, min_vel_rightwheel]
        u_max = [max_vel_leftwheel, max_vel_rightwheel]
        
        # Cost function weights
        Q = ca.diag([100.0, 100.0, 1.0])
        QN = ca.diag([100.0, 100.0, 1.0])
        R = ca.diag([1.0, 1.0])

        # Initial and reference states
        x_init = ca.MX.sym("x_init", self.nx)
        x_ref = ca.MX.sym("x_ref", self.nx)

        # OCP formulation
        X = []
        self.lbX = []
        self.ubX = []
        G = []
        self.lbG = []
        self.ubG = []
        P = []
        J = 0

        for k in range(self.N):
            x = ca.MX.sym(f"x_{k}", self.nx)
            X.append(x)
            self.lbX.extend(x_min)
            self.ubX.extend(x_max)

            G.append(
                x - x_init
                if k == 0
                else x - self.robot_dynamics(x_prev, u_prev, self.dt)  # noqa: F821
            )
            self.lbG.extend(self.nx * [0])
            self.ubG.extend(self.nx * [0])

            u = ca.MX.sym(f"u_{k}", self.nu)
            X.append(u)
            self.lbX.extend(u_min)
            self.ubX.extend(u_max)

            J += (x - x_ref).T @ Q @ (x - x_ref) + u.T @ R @ u

            x_prev = x
            u_prev = u

        x = ca.MX.sym(f"x_{self.N}", self.nx)

        X.append(x)
        self.lbX.extend(x_min)
        self.ubX.extend(x_max)

        G.append(x - self.robot_dynamics(x_prev, u_prev, self.dt))
        self.lbG.extend(self.nx * [0])
        self.ubG.extend(self.nx * [0])

        J += (x - x_ref).T @ QN @ (x - x_ref)

        # NLP parameters
        P.extend([x_init, x_ref])

        # NLP formulation
        nlp = {"f": J, "x": ca.vertcat(*X), "g": ca.vertcat(*G), "p": ca.vertcat(*P)}
        casadi_tmp_dir = "casadi_tmp/"
        os.makedirs(casadi_tmp_dir, exist_ok=True)
        opts = {
            "ipopt": {
                "print_level": 5,
                "max_wall_time": 1.2 * self.dt,
                "warm_start_init_point": "yes",
                "hsllib": "/usr/local/lib/libcoinhsl.so",
                "linear_solver": "ma57", # "ma57" or "mumps"
            },
            "jit": True,
            "jit_options": {
                "flags": ["-O3", "-ffast-math"],
                "verbose": True,
                "compiler": "gcc",
                "directory": casadi_tmp_dir,
                "cleanup": False,
            }
        }

        self.solver = ca.nlpsol("solver", "ipopt", nlp, opts)

        [os.remove(filename) for filename in glob.glob("*.c")]
        shutil.rmtree(casadi_tmp_dir, ignore_errors=True)

        # Initialize variables
        self.x_opt = np.zeros(len(self.lbX))
        self.lam_x_opt = np.zeros(len(self.lbX))
        self.lam_g_opt = np.zeros(len(self.lbG))

    def robot_dynamics(self, x, u, dt):
        v_left, v_right = u[0], u[1]
        v = wheel_radius * (v_left + v_right) / 2.0
        omega = wheel_radius * (v_right - v_left) / distance_between_wheels

        x_next = ca.vertcat(
            x[0] + v * ca.cos(x[2]) * dt,
            x[1] + v * ca.sin(x[2]) * dt,
            x[2] + omega * dt
        )
        return x_next

    def solve(self, x_init, x_ref):
        P = []
        P.append(x_init)
        P.append(x_ref)
        P = ca.vertcat(*P)

        opt_sol = self.solver(
            x0=self.x_opt,
            lam_x0=self.lam_x_opt,
            lam_g0=self.lam_g_opt,
            lbx=self.lbX,
            ubx=self.ubX,
            lbg=self.lbG,
            ubg=self.ubG,
            p=P,
        )

        self.x_opt = opt_sol["x"].full().flatten()
        self.lam_x_opt = opt_sol["lam_x"]
        self.lam_g_opt = opt_sol["lam_g"]

        u_opt = self.x_opt[self.nx + 0:self.nx + 2]
        return u_opt

In [ ]:
to_mpc = TO_MPC()

In [ ]:
to_mpc.solver()

In [ ]:
u_opt = to_mpc.solve(start, goal)